In [5]:
import os
import json
import glob
import pandas as pd
import mne
import numpy as np
import pickle

from saving_utils import path_to_names
from features_EOC import features_EOC
from features_EOS import features_EOS
from features_Pred import features_Pred
from features_slope import features_slope
from features_DFA import features_DFA
from features_AVC import features_AVC
from features_avc_std_dist import features_avc_std_dist

# ========== CONFIG ========== #
results_table_path = 'results/summary.csv'
results_dict_dir = 'results/details/'
os.makedirs(results_dict_dir, exist_ok=True)

paths = glob.glob('/Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/10s/*.fif')
paths.sort()

MAX_TRIALS = 2
MAX_S = 20

# Flags to enable/disable analyses
RUN_EOC = False
RUN_EOS = False
RUN_PRED = False
RUN_SLOPE = False
RUN_DFA = False
RUN_AVC = True
RUN_STD_DIST = False

# ========== FUNCTION ========== #
def update_results_table(path, row_dict, dict_outputs=None):
    # Load or create table
    if os.path.exists(results_table_path):
        df = pd.read_csv(results_table_path)
    else:
        df = pd.DataFrame()

    # Get identifier for the current file
    path_id = os.path.basename(path)
    row_dict['path'] = path_id  # Ensure path is included

    # ✅ Ensure 'path' column exists in df
    if 'path' not in df.columns:
        df['path'] = pd.NA  # or np.nan

    if path_id in df['path'].values:
        for key, value in row_dict.items():
            if key not in df.columns:
                df[key] = 'NA'
            if value != 'NA':
                df.loc[df['path'] == path_id, key] = value
    else:
        for key in row_dict:
            if key not in df.columns:
                df[key] = 'NA'
        df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)

    df.to_csv(results_table_path, index=False)

    # Save detailed dictionary outputs
    if dict_outputs:
        pkl_path = os.path.join(results_dict_dir, f"{path_id}.pkl")

        # Load existing data if it exists
        if os.path.exists(pkl_path):
            with open(pkl_path, 'rb') as f:
                existing_data = pickle.load(f)
        else:
            existing_data = {}

        # Update with new values
        existing_data.update(dict_outputs)

        # Save the updated version
        with open(pkl_path, 'wb') as f:
            pickle.dump(existing_data, f)




for path in paths:
    print(f"\n>>> Processing {os.path.basename(path)}")
    row_data, dict_data = {}, {}

    # Load Epoch + Raw
    mne_epochs_raw = mne.read_epochs(path, preload=True)
    mne_epochs_32 = mne_epochs_raw.copy()
    mne_epochs_32.interpolate_bads(reset_bads=True).pick_types(eeg=True)
    mne_epochs_raw.pick_types(eeg=True)

    bad_chans = mne_epochs_raw.info['bads']
    all_chans = mne_epochs_32.ch_names
    good_chans = [ch for ch in all_chans if ch not in bad_chans]
    channel_indices = {name: idx for idx, name in enumerate(all_chans)}
    bad_indices = [channel_indices[ch] for ch in bad_chans]
    good_indices = [channel_indices[ch] for ch in good_chans]

    path_raw = path.replace('_epo.fif', '_raw.fif').replace('10s', 'raw')
    raw_raw = mne.io.read_raw_fif(path_raw, preload=True)
    raw_32 = raw_raw.copy()
    raw_32.interpolate_bads(reset_bads=True).pick_types(eeg=True)
    raw_raw.pick_types(eeg=True)

    # Store metadata
    control, sub, day, condition, num_bad_channels, n_elem_raw, length_raw, n_epochs_10, length_10, n_epochs_3, length_3 = path_to_names(path)
    row_data.update({
        'control': control, 'sub': sub, 'day': day, 'condition': condition,
        'num_bad_channels': num_bad_channels, 'n_elem_raw': n_elem_raw,
        'length_raw': length_raw, 'n_epochs_10': n_epochs_10, 'length_10': length_10,
        'n_epochs_3': n_epochs_3, 'length_3': length_3
    })

    fbands = [[1, 45], [1, 4], [4, 8], [8, 13], [13, 30], [30, 45]]

    # ========== EOC ========== #
    if RUN_EOC:
        try:
            K_median, K_median_interpolated, Freq, Nopeak, eoc_results, eoc_results_interpolated = features_EOC(
                mne_epochs_32, k_type='flex', hfrequ=None, max_trials=MAX_TRIALS,
                bad_indices=bad_indices, good_indices=good_indices)
            row_data.update({
                'K_median': K_median, 'K_median_interpolated': K_median_interpolated,
                'Freq': Freq, 'Nopeak': Nopeak
            })
            dict_data.update({
                'eoc_results': eoc_results,
                'eoc_results_interpolated': eoc_results_interpolated
            })
        except Exception as e:
            print(f"[EOC] Error: {e}")

    # ========== EOS ========== #
    if RUN_EOS:
        try:
            PCF_mean, OR_mean, eos_results = features_EOS(mne_epochs_32, minfreq=1, maxfreq=10, fs=256, max_trials=MAX_TRIALS)
            row_data.update({'PCF_mean': PCF_mean, 'OR_mean': OR_mean})
            dict_data['eos_results'] = eos_results
        except Exception as e:
            print(f"[EOS] Error: {e}")

    # ========== PRED ========== #
    if RUN_PRED:
        try:
            vals = features_Pred(mne_epochs_32, lfreq=0.5, hfreq=40, fs=256, max_trials=MAX_TRIALS, bad_indices=bad_indices)
            names = ['Lyaps_max', 'Dims_mean', 'Ent_mean', 'LZC_mean', 'KDF_mean', 'Lyaps_max_interpolated', 'Dims_mean_interpolated', 'Ent_mean_interpolated', 'LZC_mean_interpolated', 'KDF_mean_interpolated']
            row_data.update({name: val for name, val in zip(names, vals[:10])})
            dict_data['pred_results'] = vals[10]
            dict_data['pred_results_interpoalted'] = vals[11]
        except Exception as e:
            print(f"[PRED] Error: {e}")

    # ========== SLOPE ========== #
    if RUN_SLOPE:
        try:
            vals = features_slope(mne_epochs_32, lfreq=0.5, hfreq=40, fs=256, max_trials=MAX_TRIALS, bad_indices=bad_indices)
            row_data.update({
                'slope_id': vals[0], 'slope_id_interpolated': vals[1]
            })
            dict_data['slope'] = {'Slope_space_id': vals[2], 
                                  'Slope_space_id_interpoalted': vals[3],
                                  'psds_mean': vals[4],
                                  'psds_mean_interpolated': vals[5],
                                  'psds': vals[6],
                                  'psds_interpoalted': vals[7],
                                  'freqs': vals[8],
                                  }
        except Exception as e:
            print(f"[SLOPE] Error: {e}")

    # ========== DFA ========== #
    if RUN_DFA:
        for fband in fbands:
            try:
                HURST_FH, HURST_DFA, results, HURST_FH_int, HURST_DFA_int, results_interpolated = features_DFA(
                    raw_32, lfreq=fband[0], hfreq=fband[1], fs=256, max_s=MAX_S, bad_indices=bad_indices)
                
                band_name = f"{fband[0]}-{fband[1]}Hz"
                row_data[f'HURST_FH_{band_name}'] = HURST_FH
                row_data[f'HURST_DFA_{band_name}'] = HURST_DFA
                row_data[f'HURST_FH_interpolated_{band_name}'] = HURST_FH_int
                row_data[f'HURST_DFA_interpolated_{band_name}'] = HURST_DFA_int

                dict_data[f'results_DFA_{band_name}'] = results
                dict_data[f'results_DFA_int_{band_name}'] = results_interpolated


            except Exception as e:
                print(f"[DFA {fband}] Error: {e}")

    # ========== AVC ========== #
    if RUN_AVC:
        try:
            out, avls = features_AVC(raw_32, bin_threshold=0.0005, max_iei=0.2, fs=256, max_s=MAX_S, lfreq=0.5, hfreq=40)
            for key, value in out.items():
                if isinstance(value, list) and len(value) > 0:
                    row_data[key] = value[0]
                else:
                    row_data[key] = 'NA'

        except Exception as e:
            print(f"[AVC] Error: {e}")

    # ========== STD_DIST ========== #
    if RUN_STD_DIST:
        try:
            part_hist_x, part_hist_y, part_hist_x_raw, part_hist_y_raw = features_avc_std_dist(raw_32, max_s=MAX_S, fs=256, lfreq=0.5, hfreq=40)
            dict_data['part_hist_x'] = part_hist_x
            dict_data['part_hist_y'] = part_hist_y
            dict_data['part_hist_x_raw'] = part_hist_x_raw
            dict_data['part_hist_y_raw'] = part_hist_y_raw

        except Exception as e:
            print(f"[STD_DIST] Error: {e}")


    # SAVE RESULTS
    update_results_table(path, row_data, dict_outputs=dict_data)



>>> Processing 0_00_01_jhana_02_026_1253_100_1000_406_1218_epo.fif
Reading /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/10s/0_00_01_jhana_02_026_1253_100_1000_406_1218_epo.fif ...


    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Opening raw data file /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/raw/0_00_01_jhana_02_026_1253_100_1000_406_1218_raw.fif...
    Range : 512 ... 321286 =      2.000 ...  1255.023 secs
Ready.
Reading 0 ... 320774  =      0.000 ...  1253.023 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of rad

Assuming nested distributions


number of elements: 20

>>> Processing 0_00_01_mindf_00_017_1384_120_1200_444_1332_epo.fif
Reading /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/10s/0_00_01_mindf_00_017_1384_120_1200_444_1332_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Opening raw data file /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/raw/0_00_01_mindf_00_017_1384_120_1200_444_1332_raw.fif...


/opt/anaconda3/lib/python3.8/site-packages/powerlaw.py:1615: RuntimeWarning: invalid value encountered in divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
Assuming nested distributions
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: divide by zero encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: invalid value encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
<ipython-input-5-7d7f83f4ab38>:96: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  mne_epochs_32.interpolate_bads(reset_bads=True).pick_types(eeg=True)


...

/opt/anaconda3/lib/python3.8/site-packages/powerlaw.py:1615: RuntimeWarning: invalid value encountered in divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
Assuming nested distributions
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: divide by zero encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: invalid value encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
Assuming nested distributions


number of elements: 28

>>> Processing 0_01_04_jhan4_03_003_0305_028_0280_101_0303_epo.fif
Reading /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/10s/0_01_04_jhan4_03_003_0305_028_0280_101_0303_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
28 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Opening raw data file /Users/jonasmago/PhD_code_data/github/eeg_jhana/notebooks/hand_cleaning/ALL/raw/0_01_04_jhan4_03_003_0305_028_0280_101_0303_raw.fif...
    Range : 3

/opt/anaconda3/lib/python3.8/site-packages/powerlaw.py:1615: RuntimeWarning: invalid value encountered in divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
Assuming nested distributions
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: divide by zero encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
/opt/anaconda3/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: invalid value encountered in divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
Assuming nested distributions


number of elements: 29


KeyboardInterrupt: 